In [7]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, precision_recall_fscore_support, confusion_matrix
from nltk.stem.snowball import SnowballStemmer
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import seaborn as sn
from sklearn import datasets
from nltk.tokenize import RegexpTokenizer
from gensim.sklearn_api.w2vmodel import W2VTransformer
from gensim.sklearn_api.d2vmodel import D2VTransformer
from nltk.corpus import stopwords
from ast import literal_eval
import xlrd
import openpyxl

In [2]:
def row(key, scores, params):
            d = {
                 'estimator': key,
                 'mean_score': np.mean(scores),
                 'f1-scores': scores,
            }
            return pd.Series({**params,**d})
tokenizer = RegexpTokenizer(r'\w+')

In [3]:
%%time
df = pd.read_csv("Handelingen.csv", index_col=0)
Partij = df.partij
sbs = SnowballStemmer("dutch")
listTekst = df.tekst.apply(lambda x: [sbs.stem(t) for t in tokenizer.tokenize(x)])
normalTekst = df.tekst.apply(lambda x: ' '.join([sbs.stem(t) for t in tokenizer.tokenize(x)]))

Wall time: 3min 11s


In [4]:
df = pd.read_excel('Models.xlsx', index_col=0)
df3 = df.loc[df.DOEN == True]
models = {}

for x,y in zip(df3.Classifier,df3.PIPELINE):
    exec(compile("a="+y,'','exec'))
    models[x] = Pipeline(a)    
params = {}

for clf in set(df3.Classifier):
    test = df3.loc[df3.Classifier == clf]
    test = test.loc[test.DOEN == True]
    test = test.dropna(axis=1, how='all')
    test= test.drop([col for col in test.columns if "__" not in col], axis =1)
    test = test.to_dict(orient='list')
    testdict = {}
    for x,y in test.items():
        a=literal_eval(y[0])
        #exec(compile("a="+y[0],'','exec'))
        testdict[x] = a   
    params[clf] = testdict
rawdict = {x:y for x,y in zip(df3.Classifier,df3.RAW)}

In [ ]:
for name,pipe in models.items():
    if rawdict[name]:
         Tekst = list(listTekst)
    else:
        Tekst = list(normalTekst)
    gs = GridSearchCV(pipe, params[name], cv=5, scoring='f1_weighted')
    gs.fit(Tekst, Partij)
    rows = [row(name, gsc.cv_validation_scores, gsc.parameters) for gsc in gs.grid_scores_]
    df2 = pd.concat(rows, axis=1).T
    pd.concat([df2,pd.read_csv('Scores.csv', index_col=0)], ignore_index=True).sort_values(['mean_score'], ascending=False).to_csv('Scores.csv')
    df.loc[df.Classifier==name,"DOEN"]=False
    df.to_excel('Models.xlsx')
    
#https://stackoverflow.com/questions/46735847/save-best-params-in-gridsearch-in-a-pandas-dataframe
# https://stackoverflow.com/questions/36271413/pandas-merge-nearly-duplicate-rows-based-on-column-value?rq=1